## 7 畳み込みネットワーク

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

## 7.4 Convolution / Poolingレイヤの実装

### 7.4.1 4次元配列

In [3]:
import numpy as np
x = np.random.rand(10, 1, 28, 28) # ランダムにデータを生成
x.shape

(10, 1, 28, 28)

In [4]:
x[0].shape # 1つ目のデータにアクセス. (1, 28, 28)

(1, 28, 28)

In [5]:
x[1].shape # 2つ目のデータにアクセス. (1, 28, 28)

(1, 28, 28)

In [6]:
x[0, 0] # 1チャンネル目の空間データにアクセス, もしくはx[0][0]

array([[0.50314773, 0.02010111, 0.29824062, 0.41081747, 0.1104862 ,
        0.71809124, 0.99714848, 0.46179259, 0.42546508, 0.75866732,
        0.53650481, 0.62224765, 0.39140766, 0.21440149, 0.64959127,
        0.47155295, 0.13775211, 0.27653699, 0.08869034, 0.11548296,
        0.96230659, 0.63627579, 0.22067741, 0.77279314, 0.22321556,
        0.96208114, 0.90043464, 0.12921593],
       [0.12935989, 0.36268397, 0.74365969, 0.26779202, 0.55628933,
        0.798729  , 0.16749911, 0.39823327, 0.6961941 , 0.85125801,
        0.84553473, 0.58123161, 0.09306764, 0.70251515, 0.74076846,
        0.26890364, 0.73307079, 0.3331027 , 0.47342265, 0.88940869,
        0.47005862, 0.22758157, 0.78452763, 0.17309371, 0.98530086,
        0.36313938, 0.4865346 , 0.5114959 ],
       [0.21332189, 0.62454135, 0.93820707, 0.90796258, 0.68486689,
        0.40830686, 0.27907715, 0.89228979, 0.51531886, 0.30110219,
        0.50768791, 0.25833619, 0.29597463, 0.53797972, 0.25921112,
        0.17599448, 0.7088

### 7.4.3 Convolutionレイヤの実装

In [7]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド
    pad : パディング

    Returns
    -------
    col : 2次元配列
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [8]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
coll = im2col(x1, 5, 5, stride = 1, pad = 0)
print(coll.shape) # (9, 75)

x2 = np.random.rand(10, 3, 7, 7) # 10個のデータ
col2 = im2col(x2, 5, 5, stride = 1, pad = 0)
print(col2.shape) #(90, 75)

(9, 75)
(90, 75)


In [9]:
class Convolution: 
    def __init__(self, W, b, stride = 1, pad = 0): 
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x): 
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # フィルターの展開
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(o, 3, 1, 2)
        
        return out

In [10]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    col :
    input_shape : 入力データの形状（例：(10, 1, 28, 28)）
    filter_h :
    filter_w
    stride
    pad

    Returns
    -------

    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [11]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 中間データ（backward時に使用）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 重み・バイアスパラメータの勾配
        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

### 7.4.4 Poolingレイヤの実装

In [12]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
#        self.x = None
#        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 展開(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

#        arg_max = np.argmax(col, axis=1)
        # 最大値(2)
        out = np.max(col, axis=1)
        # 最大値(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

#        self.x = x
#        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx

## 7.5 CNNの実装

In [13]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class SimpleConvNet:
    """単純なConvNet

    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 入力サイズ（MNISTの場合は784）
    hidden_size_list : 隠れ層のニューロンの数のリスト（e.g. [100, 100, 100]）
    output_size : 出力サイズ（MNISTの場合は10）
    activation : 'relu' or 'sigmoid'
    weight_init_std : 重みの標準偏差を指定（e.g. 0.01）
        'relu'または'he'を指定した場合は「Heの初期値」を設定
        'sigmoid'または'xavier'を指定した場合は「Xavierの初期値」を設定
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """損失関数を求める
        引数のxは入力データ、tは教師ラベル
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """勾配を求める（数値微分）

        Parameters
        ----------
        x : 入力データ
        t : 教師ラベル

        Returns
        -------
        各層の勾配を持ったディクショナリ変数
            grads['W1']、grads['W2']、...は各層の重み
            grads['b1']、grads['b2']、...は各層のバイアス
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """勾配を求める（誤差逆伝搬法）

        Parameters
        ----------
        x : 入力データ
        t : 教師ラベル

        Returns
        -------
        各層の勾配を持ったディクショナリ変数
            grads['W1']、grads['W2']、...は各層の重み
            grads['b1']、grads['b2']、...は各層のバイアス
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 処理に時間のかかる場合はデータを削減 
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# パラメータの保存
network.save_params("params.pkl")
print("Saved Network Parameters!")

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.299748755245998
=== epoch:1, train acc:0.363, test acc:0.389 ===
train loss:2.297429905722912
train loss:2.290090204182562
train loss:2.2881329416864076
train loss:2.2764572697026106
train loss:2.272383712675069
train loss:2.2575599988509585
train loss:2.246611064239028
train loss:2.2035093834255304
train loss:2.1997930297992205
train loss:2.1809009817342564
train loss:2.151497414651663
train loss:2.083762059540119
train loss:2.0164656134103556
train loss:1.9862587965607255
train loss:1.9307814581916114
train loss:1.847066981292377
train loss:1.7366085159148228
train loss:1.7607630491835529
train loss:1.6421829172276878
train loss:1.603955351891702
train loss:1.4985054607703479
train loss:1.372026151904675
train loss:1.3626628363549032
train loss:1.2326938825102938
train loss:1.2752914833132858
train loss:1.047200374977956
train loss:1.0483299692214585
train loss:0.9850571050985794
train loss:0.9628816176423968
train loss:0.9512399063760506
train loss:0.8864245991415064
tr

train loss:0.27430715059890276
train loss:0.2736885242494234
train loss:0.2915911685430388
train loss:0.3073572495023866
train loss:0.208381151101929
train loss:0.2304389180946482
train loss:0.19925289732568083
train loss:0.17408540706493844
train loss:0.1658249418662356
train loss:0.21396794070984318
train loss:0.23404868856286423
train loss:0.22823600843743097
train loss:0.10670614956698754
train loss:0.15711913711005537
train loss:0.18915402609886223
train loss:0.22986550246640217
train loss:0.16117388462177046
train loss:0.23681508602805304
train loss:0.303725578007057
train loss:0.1687970033866992
train loss:0.14670405315210824
train loss:0.15529020068618402
train loss:0.11130703814132663
train loss:0.2642077691299533
train loss:0.13963776263052402
train loss:0.17261384120598744
train loss:0.2049858389491234
train loss:0.19816915709077662
train loss:0.09547630755435592
train loss:0.1501308827132404
train loss:0.2649214371307962
train loss:0.15583812024856852
train loss:0.240362990

train loss:0.09573910266044022
train loss:0.07710905697406972
train loss:0.06566829275539778
train loss:0.10419152457196094
train loss:0.10407874994569367
train loss:0.1351523995034106
train loss:0.11253099339785946
train loss:0.04293178148075612
train loss:0.07829065086837925
train loss:0.10425507193470829
train loss:0.1306867233910233
train loss:0.06274352379030225
train loss:0.09618449561689507
train loss:0.13850299297643004
train loss:0.050596757185201964
train loss:0.0802167256588599
train loss:0.10346553364864185
train loss:0.25072701716516654
train loss:0.12486352346802836
train loss:0.08099853276637979
train loss:0.048215122460969095
train loss:0.1072211278544525
train loss:0.10320827699505986
train loss:0.18094974933692304
train loss:0.13324361973206425
train loss:0.1352193178489119
train loss:0.08669835368039762
train loss:0.12769448028496222
train loss:0.1744987562668713
train loss:0.08175873366444115
train loss:0.052641643167760904
train loss:0.09867233451256523
train loss:

train loss:0.06732237749135316
train loss:0.05627209903821813
train loss:0.13277094998476519
train loss:0.059631999329679664
train loss:0.051258809813370394
train loss:0.08762786094044937
train loss:0.04695982233347923
train loss:0.12132064787966079
train loss:0.10469589452639987
train loss:0.11789011129637438
train loss:0.04913659965838236
train loss:0.05998201286722499
train loss:0.05623625924728975
train loss:0.07626201247515939
train loss:0.040286777668407464
train loss:0.039813761231417194
train loss:0.08861364593868254
train loss:0.06513237978397025
train loss:0.06504251254056574
train loss:0.14822046306588235
train loss:0.17538751232836425
train loss:0.08188873809137029
train loss:0.1477413699509693
train loss:0.03736729559617044
train loss:0.20280466092152546
train loss:0.0654982948444053
train loss:0.1057318913621579
train loss:0.06808827830019287
train loss:0.0473155752585366
train loss:0.0643015569256174
train loss:0.03876871471192302
train loss:0.13420478502652716
train los

train loss:0.034287145705486924
train loss:0.03758502634243471
train loss:0.04859641082513006
train loss:0.12538735212263646
train loss:0.07304361333198532
train loss:0.03331761824763707
train loss:0.04796085672943792
train loss:0.0669164600662477
train loss:0.017567281958342885
train loss:0.029700157965126994
train loss:0.03606854073705594
train loss:0.08787208331590929
train loss:0.060781190853389976
train loss:0.05987886408717143
train loss:0.024535727033010105
train loss:0.05169223463249456
train loss:0.06492502281842144
train loss:0.06337981618892158
train loss:0.05126414563948934
train loss:0.06345307271980954
train loss:0.05055719655066536
train loss:0.08544690999650342
train loss:0.028471678758486857
train loss:0.04925899411010527
train loss:0.09895539573076073
train loss:0.09196936738766523
train loss:0.04593974888151938
train loss:0.10915141861961597
train loss:0.05415469751772748
train loss:0.06511155641682108
train loss:0.01948158735199941
train loss:0.0818115353948724
trai

...

train loss:0.007122710638048969
train loss:0.017813767155393335
train loss:0.056672522182631455
train loss:0.0069031284349590025
train loss:0.012763407742927519
train loss:0.026855562288827538
train loss:0.010291268927110771
train loss:0.05281306439850144
train loss:0.02266140006785408
train loss:0.0028031830909425086
train loss:0.05530838199386156
train loss:0.04597514613537713
train loss:0.0013253116562884101
train loss:0.02046688407888946
train loss:0.02082691149944871
train loss:0.005764640905533833
train loss:0.009420700175032229
train loss:0.03722880794479472
train loss:0.008021468971412671
train loss:0.025794436490848813
train loss:0.003588150105988461
train loss:0.042627834496714846
train loss:0.04348320018233934
train loss:0.005764291293976712
train loss:0.06555017066680045
train loss:0.0020286834376732187
train loss:0.033003801656085775
train loss:0.00261470765054601
train loss:0.009337451171349808
train loss:0.002331935779090856
train loss:0.018647179778092018
train loss:0.0

train loss:0.011724302834133447
train loss:0.006174170756016328
train loss:0.025631038768424822
train loss:0.0028946247704944384
train loss:0.005273420272722121
train loss:0.010048939418326774
train loss:0.004470825161994893
train loss:0.0342844114426036
train loss:0.019004466275270564
train loss:0.00405686429988474
train loss:0.012405749776914539
train loss:0.018583839331376167
train loss:0.022871444284568818
train loss:0.0029086607205536246
train loss:0.0024711745321747635
train loss:0.024815196629804612
train loss:0.04473915986643057
train loss:0.006553916719616772
=== epoch:8, train acc:0.991, test acc:0.986 ===
train loss:0.0007361969360463469
train loss:0.005752088399444504
train loss:0.00725954578663467
train loss:0.009132478222948248
train loss:0.02856561857268436
train loss:0.0014382441573503083
train loss:0.003920129351369683
train loss:0.024587119688405195
train loss:0.0044656587756079814
train loss:0.13088619221505213
train loss:0.01356187328569467
train loss:0.025694563370

train loss:0.0040870470927650586
train loss:0.003874120835286527
train loss:0.01386092234863306
train loss:0.005689295277175496
train loss:0.01737337518286407
train loss:0.008173623101288562
train loss:0.003037940128926202
train loss:0.01873922644251929
train loss:0.018041640716616236
train loss:0.009914392693613931
train loss:0.013508800049778925
train loss:0.017005475987413688
train loss:0.014470955456169847
train loss:0.0035131013367344726
train loss:0.003940749291563673
train loss:0.05264882773569653
train loss:0.006752075784813085
train loss:0.013217647056774043
train loss:0.019690985365968593
train loss:0.04470945898069733
train loss:0.008381937657594409
train loss:0.002795782815663373
train loss:0.011107457881362161
train loss:0.03049032279617452
train loss:0.03291664010240462
train loss:0.005753492002410322
train loss:0.01966409147060851
train loss:0.0041403078723407285
train loss:0.002258908612866405
train loss:0.008863664527720819
train loss:0.002153722719723612
train loss:0.

train loss:0.001510427888066737
train loss:0.02892915885793053
train loss:0.004110747115967729
train loss:0.010442836375874898
train loss:0.011875270430071633
train loss:0.005481393410313927
train loss:0.009071863690667968
train loss:0.00470503740784601
train loss:0.004126599651472907
train loss:0.011952653341901678
train loss:0.0068198901071365205
train loss:0.02035448492404971
train loss:0.00884317582926594
train loss:0.024457143065716194
train loss:0.020944620524130587
train loss:0.0009319890833455925
train loss:0.004620850124145213
train loss:0.006366845366391525
train loss:0.04026268320107864
train loss:0.0038993728824061524
train loss:0.006233207441095543
train loss:0.0018398753769265155
train loss:0.00583716295466149
train loss:0.012271479165426084
train loss:0.0035998942641957744
train loss:0.004067352993648441
train loss:0.008462876719413608
train loss:0.012023282491580136
train loss:0.01567815530211123
train loss:0.006970304707076776
train loss:0.017768116114051945
train loss

train loss:0.015868012465687094
train loss:0.0013325867020251433
train loss:0.03056002840803142
train loss:0.00987981717317511
train loss:0.0034877092388814583
train loss:0.003445345917825227
train loss:0.003186665195103755
train loss:0.0030241871124656616
train loss:0.03464921290780745
train loss:0.0008415270701208962
train loss:0.001345889758047253
train loss:0.010357839995908873
train loss:0.018120267162504814
train loss:0.006248107023525099
train loss:0.009286807376967292
train loss:0.001851955576626075
train loss:0.0026930130345837427
train loss:0.0031602744537940625
train loss:0.002200433488011448
train loss:0.00992806267509327
train loss:0.020654710793131175
train loss:0.0285131466180462
train loss:0.00991062519401294
train loss:0.004054257155172329
train loss:0.006159395096076152
train loss:0.0025242918247859974
train loss:0.003970914156680455
train loss:0.005041643236320436
train loss:0.004762011495693098
train loss:0.0007236366585385201
train loss:0.020809316141857978
train l